In [1]:
! pip install -q gliner
! pip install -q accelerate -U

    torch (>=1.7.*)
           ~~~~~~^
ERROR: Could not install packages due to an OSError: [WinError 5] Отказано в доступе: 'C:\\Users\\admin\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\onnxruntime\\capi\\onnxruntime_providers_shared.dll'
Consider using the `--user` option or check the permissions.

    torch (>=1.7.*)
           ~~~~~~^
  You can safely remove it manually.


In [2]:
# download data
# ! wget https://huggingface.co/datasets/urchade/synthetic-pii-ner-mistral-v1/resolve/main/data.json

In [1]:
import json
import random

In [3]:
train_path = "gliner-jkh.json"

with open(train_path, "r") as f:
    data = json.load(f)

print('Dataset size:', len(data))

random.shuffle(data)
print('Dataset is shuffled...')

train_dataset = data[:int(len(data)*0.9)]
test_dataset = data[int(len(data)*0.9):]

print('Dataset is splitted...')

Dataset size: 4250
Dataset is shuffled...
Dataset is splitted...


In [4]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"

import torch
from gliner import GLiNERConfig, GLiNER
from gliner.training import Trainer, TrainingArguments
from gliner.data_processing.collator import DataCollatorWithPadding, DataCollator
from gliner.utils import load_config_as_namespace
from gliner.data_processing import WordsSplitter, GLiNERDataset

In [5]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

model = GLiNER.from_pretrained("urchade/gliner_multi-v2.1")
# model = GLiNER.from_pretrained("models/checkpoint-3006")


config.json not found in C:\Users\admin\Desktop\semifinal\gliner_train\models\checkpoint-3006
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
# use it for better performance, it mimics original implementation but it's less memory efficient
data_collator = DataCollator(model.config, data_processor=model.data_processor, prepare_labels=True)

In [7]:
# Optional: compile model for faster training
model.to(device)
print("done")

done


In [8]:
# calculate number of epochs
num_steps = 4500
batch_size = 15
data_size = len(train_dataset)
num_batches = data_size // batch_size
num_epochs = max(1, num_steps // num_batches)

training_args = TrainingArguments(
    output_dir="models",
    learning_rate=5e-6,
    weight_decay=0.01,
    others_lr=1e-5,
    others_weight_decay=0.01,
    lr_scheduler_type="linear", #cosine
    warmup_ratio=0.1,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    focal_loss_alpha=0.75,
    focal_loss_gamma=2,
    num_train_epochs=num_epochs,
    evaluation_strategy="steps",
    save_steps = 500,
    save_total_limit=10,
    dataloader_num_workers = 0,
    use_cpu = False,
    report_to="none",
    )

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=model.data_processor.transformer_tokenizer,
    data_collator=data_collator,
)

trainer.train()

c:\Users\admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/4335 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'loss': 7.3539, 'grad_norm': 227.15304565429688, 'learning_rate': 9.830812612150732e-06, 'epoch': 1.96}


  0%|          | 0/29 [00:00<?, ?it/s]

{'eval_loss': 52.56298828125, 'eval_runtime': 44.6003, 'eval_samples_per_second': 9.529, 'eval_steps_per_second': 0.65, 'epoch': 1.96}


RuntimeError: [enforce fail at inline_container.cc:603] . unexpected pos 1475256640 vs 1475256528

In [12]:
trained_model = GLiNER.from_pretrained("models/checkpoint-500", load_tokenizer=True)


text = """
из п\з д200  Без хвс Нариманова 1,1г,3,5, ч/д в границах: Кооперат.- пер.Новикова- Лермонтова- Нариманова
"""

# Labels for entity prediction
labels = ["address"] # for v2.1 use capital case for better performance

# Perform entity prediction
entities = trained_model.predict_entities(text, labels, threshold=0.5)

# Display predicted entities and their labels
for entity in entities:
    print(entity["text"], "=>", entity["label"])

Кооперат => address
Новикова- Лермонтова- Нариманова => address
